In [1]:
import numpy as np
from scipy import signal
import pickle
import os
import sys

In [2]:
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/Eating/myutils' if 'C:' in os.getcwd() else './myutils'
sys.path.append(util_path)
import my_file_utils as mfileu

In [ ]:
def mean_magnitude(d):
    d = np.sqrt(np.sum(d*d, axis=1))
    return np.mean(d)

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def zero_cross(d):
    x = (d[:-1]*d[1:]<0)
    zcr = np.sum(x)
    x = np.argwhere(x==True)
    #print(x.reshape((1,-1)))
    std = np.std(x[1:]-x[:-1])    
    return zcr, std    

In [ ]:
def get_features_one_window(d):
    t, accel, gyro = d[:, 0], d[:, 1:4], d[:, 4:7]
    
    accel_mean = np.mean(accel, axis=0)
    gyro_mean = np.mean(gyro, axis=0)
    gyro_mean_mag = mean_magnitude(gyro)
    accel_dv_mean_mag = mean_magnitude(accel[1:, :]-accel[:-1, :])
    
    accel_cov = np.cov(accel, rowvar=False)
    #accel_cov = np.corrcoef(accel, rowvar=False)
    accel_cov = np.array([accel_cov[0,1], accel_cov[0,2], accel_cov[1,2]])
    
    hamming_window = signal.hamming(len(accel))    
    poly_x = np.polyfit(t-t[0], accel[:, 0], deg=4, w=hamming_window)
    poly_y = np.polyfit(t-t[0], accel[:, 1], deg=4, w=hamming_window)
    poly_z = np.polyfit(t-t[0], accel[:, 2], deg=4, w=hamming_window)
    
    high_x = butter_highpass_filter(accel[:, 0], cutoff = 4, fs=16)
    high_y = butter_highpass_filter(accel[:, 1], cutoff = 4, fs=16)
    high_z = butter_highpass_filter(accel[:, 2], cutoff = 4, fs=16)
    
    accel_zcr_x, accel_zcr_std_x = zero_cross(high_x)
    accel_zcr_y, accel_zcr_std_y = zero_cross(high_y)
    accel_zcr_z, accel_zcr_std_z = zero_cross(high_z)
    
    res = np.zeros((1, 32))
    res[0, 0:3] = accel_mean
    res[0, 3:6] = gyro_mean
    res[0, 6] = gyro_mean_mag
    res[0, 7] = accel_dv_mean_mag
    res[0, 8:11] = accel_cov
    res[0, 11:16] = poly_x
    res[0, 16:21] = poly_y
    res[0, 21:26] = poly_z
    res[0, 26] = accel_zcr_x
    res[0, 27] = accel_zcr_y
    res[0, 28] = accel_zcr_z
    res[0, 29] = accel_zcr_std_x
    res[0, 30] = accel_zcr_std_y
    res[0, 31] = accel_zcr_std_z
    
    return res

In [ ]:
def get_features_one_session(data, window_size=5*16, step_size=1):
    count = (len(data)-window_size)//step_size + 1
    res = np.zeros((count, 33))
    
    print("Data sahpe: ", data.shape, "Feature shape: ", res.shape)
    for i in range(count):
        si = i*step_size        
        d = data[si:si+window_size, :]
        res[i, 0] = si
        res[i, 1:] = get_features_one_window(d)
        if i%10000==0:
            print(i)
    
    return res

In [ ]:
def get_features_all(ds):
    features = []    
    for subj in range(len(ds)):
        fsubj =[]
        for sess in range(len(ds[subj])):
            print("\nExtracing features... Subj, sess: ", subj, sess)
            d = ds[subj][sess]
            f = get_features_one_session(d)
            print("Feature shape: ", f.shape)
            fsubj.append(f)
            
        features.append(fsubj)
        
    return features

In [ ]:
def get_features(ds, subj_start, subj_end, lab_free):        
    for subj in range(subj_start, subj_end+1):        
        for sess in range(len(ds[subj])):
            print("\nExtracing features... Subj, sess: ", subj, sess)
            d = ds[subj][sess]
            f = get_features_one_session(d)            
            print("Feature shape: ", f.shape)
            mfileu.write_file('features/'+lab_free, lab_free+'_features_'+str(subj)+'_'+str(sess)+'.pkl', f)

In [ ]:
lab_free, subj_start, subj_end = sys.argv[1], int(sys.argv[2]), int(sys.argv[3])
print("Lab/free, start subj index, end subj index: ", lab_free, subj_start, subj_end)
assert lab_free in ['lab', 'free']

if lab_free=='lab':
    ds = mfileu.read_file('data', 'lab_data_combo_right.pkl')    
else:
    ds = mfileu.read_file('data', 'free_data_steven_right.pkl')

assert 0 <= subj_start <= subj_end < len(ds)
get_features(ds, subj_start, subj_end, lab_free)